In [1]:
from collections import defaultdict
from surprise import SVD
from surprise import Reader, Dataset
import pandas as pd
from surprise.model_selection import KFold
from surprise import dump
from surprise import accuracy
from surprise.model_selection import cross_validate
import pymysql
import csv
from datetime import datetime #,timedelta
import time
import surprise

##db
db = pymysql.connect(host='###', user='###', password='###' ,db='###', charset='###')

# prepare a cursor object using cursor() method
cursor = db.cursor()
# execute SQL query using execute() method.
cursor.execute("SELECT VERSION()")
# Fetch a single row using fetchone() method.
data = cursor.fetchone()
print("Database version : %s " % data)


def get_top_n(predictions, n=10):

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
    

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

train_data= pd.read_csv(r"./usertag.csv") #텍스트 태그 이미지 태그..
df=pd.DataFrame(train_data)


df['tag_id']= df['tag_id'].astype(str)
df['user_id']= df['user_id'].astype(str)

print("df.shape={}".format(df.shape))
print(df.dtypes)
print(df.head())
print(df.describe(include='all'))


#reader class using only the rating parameter
reader= Reader(rating_scale=(0.0, 5.0))

#arranging dataframe
data= Dataset.load_from_df(df[['user_id', 'tag_id', 'userTag_count']], reader)

#build full trainset and use cross validation for evaluation
trainset= data.build_full_trainset()

algo = SVD()
algo.fit(trainset)

##db
db = pymysql.connect(host='###', user='###', password='###' ,db='###', charset='###')

# prepare a cursor object using cursor() method
cursor = db.cursor()
# execute SQL query using execute() method.
cursor.execute("SELECT VERSION()")
# Fetch a single row using fetchone() method.
data = cursor.fetchone()
print("Database version : %s " % data)

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)
accuracy.rmse(predictions)
top_n = get_top_n(predictions, n=5)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    #sql= "UPDATE `cftable SET user_id='%s' cf_id='%s' rate='%s'" % (uid,uid,user_ratings)
    for (iid, rating) in user_ratings:
        print(uid,iid,rating)
        #sql= "UPDATE `cftable SET user_id='%s' cf_id='%s' rate='%s'" % (uid,iid,rating)
        sql= "INSERT INTO cftable (user_id,cf_id,rate) VALUES (%s,%s,%s)" 
        cursor.execute(sql, (int(uid),int(iid),float(rating)))
        db.commit()
        
db.close()
print("완료")

Database version : 5.7.28-0ubuntu0.18.04.4 
df.shape=(1274, 5)
userTag_id        int64
user_id          object
tag_id           object
userTag_count     int64
time             object
dtype: object
   userTag_id user_id tag_id  userTag_count              time
0           1       2  52076              2  2019-12-30 16:47
1           2       2  50903              2  2019-12-30 16:47
2           3       2  50375              2  2019-12-30 16:47
3           4       2  20371              2  2019-12-30 16:47
4           5       2  52071              2  2019-12-30 16:47
         userTag_id user_id tag_id  userTag_count              time
count   1274.000000    1274   1274    1274.000000              1274
unique          NaN       5    870            NaN                 8
top             NaN       3  30817            NaN  2019-12-30 16:52
freq            NaN     294      4            NaN               296
mean     637.500000     NaN    NaN       1.191523               NaN
std      367.916431    

In [2]:
#pred = algo.predict("1", "10303", r_ui=0, verbose=True)
#pred = algo.predict("2", "11363", r_ui=0, verbose=True)
#pred = algo.predict("3", "52954", r_ui=0, verbose=True)
#pred = algo.predict("4", "10151", r_ui=0, verbose=True)
#pred = algo.predict("5", "31169", r_ui=0, verbose=True)

user: 1          item: 10303      r_ui = 0.00   est = 1.85   {'was_impossible': False}
user: 2          item: 11363      r_ui = 0.00   est = 2.13   {'was_impossible': False}
user: 3          item: 52954      r_ui = 0.00   est = 1.82   {'was_impossible': False}
user: 4          item: 10151      r_ui = 0.00   est = 1.50   {'was_impossible': False}
user: 5          item: 31169      r_ui = 0.00   est = 1.64   {'was_impossible': False}


In [3]:
#pred = algo.predict("1", "10303", r_ui=0, verbose=True)
#pred = algo.predict("2", "50458", r_ui=4, verbose=True)
#pred = algo.predict("3", "52954", r_ui=0, verbose=True)
#pred = algo.predict("4", "50458", r_ui=3, verbose=True)
#pred = algo.predict("5", "31169", r_ui=0, verbose=True)

user: 1          item: 10303      r_ui = 0.00   est = 1.85   {'was_impossible': False}
user: 2          item: 50458      r_ui = 4.00   est = 2.10   {'was_impossible': False}
user: 3          item: 52954      r_ui = 0.00   est = 1.82   {'was_impossible': False}
user: 4          item: 50458      r_ui = 3.00   est = 1.71   {'was_impossible': False}
user: 5          item: 31169      r_ui = 0.00   est = 1.64   {'was_impossible': False}


In [4]:
#특정아이템에 대한 유저의 응답을 통해 예측이 잘 되었는지 확인
#import matplotlib.pyplot as plt
#%matplotlib notebook
#df.loc[df['tag_id'] == "50458"]['userTag_count'].describe()
#df.loc[df['tag_id'] == "50458"]['userTag_count'].hist()
#plt.xlabel('tag')
#plt.ylabel('Number of ratings')
#plt.title('Number of ratings about tag')
#plt.show()

count    2.000000
mean     3.500000
std      0.707107
min      3.000000
25%      3.250000
50%      3.500000
75%      3.750000
max      4.000000
Name: userTag_count, dtype: float64

In [5]:
#pred = algo.predict("1", "52969", r_ui=3, verbose=True)
#pred = algo.predict("4", "52969", r_ui=1, verbose=True)
#pred = algo.predict("5", "52969", r_ui=1, verbose=True)

user: 1          item: 52969      r_ui = 3.00   est = 1.63   {'was_impossible': False}
user: 4          item: 52969      r_ui = 1.00   est = 1.23   {'was_impossible': False}
user: 5          item: 52969      r_ui = 1.00   est = 1.30   {'was_impossible': False}


In [6]:
#result = pd.DataFrame([
#    [1, '10303', 0,1.78],
#    [2, '50458', 4,1.99],
#    [3, '52954', 0,1.89],
#    [4, '50458', 3,1.69],
#    [5, '31169', 0,1.69]],
#    columns = ['user', 'item', 'Actual rating','Computed rating'])

#result

,user,item,Actual rating,Computed rating
0,1,10303,0,1.78
1,2,50458,4,1.99
2,3,52954,0,1.89
3,4,50458,3,1.69
4,5,31169,0,1.69


In [15]:
#RMSE: 0.1743
#RMSE: 0.1344
#rmse = pd.DataFrame({' ': ['RMSE'], 
#        'Text': ['0.1743'], 
#        'Text & Image': ['0.1344']})
#rmse

,,Text,Text & Image
0,RMSE,0.1743,0.1344
